In [2]:
import numpy as np
import glob
import cv2
import os

In [9]:
def kali_n(n):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    w = 9   # 10 - 1
    h = 6   # 7  - 1
    objp = np.zeros((w*h,3), np.float32)
    objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2)
    objp = objp*18.1  # 18.1 mm
    objpoints = []
    imgpoints = []
    images_k = glob.glob(r'RoboGlueI/kalibrierung/*.png')

    i=0
    for fname in images_k[:n]:
        img = cv2.imread(fname)
        h1, w1 = img.shape[0], img.shape[1]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        u, v = img.shape[:2]
        ret, corners = cv2.findChessboardCorners(gray, (w,h),None)
        if ret == True:
            # print("i:", i)
            i = i+1
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            objpoints.append(objp)
            imgpoints.append(corners)
            cv2.drawChessboardCorners(img, (w,h), corners, ret)
            cv2.namedWindow('findCorners', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('findCorners', 640, 480)
            cv2.imshow('findCorners',img)
            cv2.waitKey(200)
    cv2.destroyAllWindows()

    ret, mtx, dist, rvecs, tvecs = \
        cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (u, v), 0, (u, v))
    return dist[0], [newcameramtx[0][0], newcameramtx[1][1], newcameramtx[0][2], newcameramtx[1][2]]

In [10]:
kali_n(3)

UnboundLocalError: local variable 'gray' referenced before assignment